https://huggingface.co/collections/google/gemma-3-release-67c6c6f89c4f76621268bb6d \
https://huggingface.co/google/gemma-3-4b-it

In [1]:
import os
import sys

In [2]:
# Set CUDA_VISIBLE_DEVICES to expose only device 0
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Set CUDA_VISIBLE_DEVICES to expose devices 0, 1, 2, and 3
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [3]:
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from PIL import Image
import requests
import torch

/data/mn27889/miniconda3/envs/path-opendata/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_id = "google/gemma-3-4b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True
).eval()

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.65s/it]


In [5]:
processor = AutoProcessor.from_pretrained(model_id)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [6]:
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "./DeepSeek-VL2/images/visual_grounding_1.jpeg"},
            {"type": "text", "text": "Describe this image in detail."}
        ]
    }
]

In [7]:
inputs = processor.apply_chat_template(messages, add_generation_prompt=True, tokenize=True,
                                        return_dict=True, return_tensors="pt")

inputs = inputs.to(model.device)

In [8]:
input_len = inputs["input_ids"].shape[-1]

In [9]:
with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=200, do_sample=False)
    generation = generation[0][input_len:]

The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/data/mn27889/miniconda3/envs/path-opendata/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:236: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
CUDAGraph supports dynamic shapes by recording a new graph for each distinct input size. Recording too many CUDAGraphs may lead to extra overhead. We have observed 51 distinct sizes. Please consider the following options for better performance: a) padding inputs to a few fixed number of shapes; or b) set torch._inductor.config.triton.cudagraph_skip_dynamic_graphs=True. Set torch._inductor.config.triton.cudagraph_dynamic_shape_warn_limit=None to silence this warning.


In [10]:
decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

Okay, here's a detailed description of the image you sent:

**Overall Impression:**

The image captures a serene and beautiful scene of two giraffes standing in a grassy field under a clear blue sky. The lighting suggests it’s either early morning or late afternoon, casting long shadows.

**Foreground:**

*   **Giraffes:** Two giraffes are the primary focus.
    *   The giraffe on the left is taller and more prominent, standing upright with its head raised high. It has a classic giraffe pattern of brown spots on a tan body.
    *   The giraffe on the right is slightly smaller and appears to be looking towards the left. It also has the distinctive spotted pattern.
*   **Grass:** The ground is covered in lush, green grass, providing a vibrant contrast to the giraffes' coats.

**Background:**

*   **Trees:** A dense line of trees forms the backdrop, creating a natural border to the field. The
